<a href="https://colab.research.google.com/github/Miguelapp10/correo_masivo_python/blob/main/correo_masivo_tienda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import time
import locale
import datetime as dt  # Para fechas (opcional)
import glob  # Para jalar todo los archivos en una carpeta
import os  # Para trabajar con rutas
import warnings  # Para evitar que salgan errores en formatos de archivo (no altera el producto)
from datetime import datetime, timedelta # Para fechas (opcional)
from io import StringIO  # Usado para definir función nueva
from pathlib import Path
import numpy as np  # Para operaciones matemáticas
import xlsxwriter  # Funcionalidad para trabajar con archivos Excel
from fpdf import FPDF  # Para crear archivos PDF
from pandas import ExcelWriter  # Para exportar tabla a Excel
from xlsx2csv import Xlsx2csv  # Usado para definir función nueva
from decimal import Decimal

import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.errors import HttpError
import io

# Usuario
usuario = r''
# Ruta
ruta = r''
# 1. Rutas base
ruta_base = r'C:\\Users\\' + usuario + r'\\Falabella\\' + ruta
## lista Personal de recolección
ruta_Tienda = ruta_base + r'\\PRD_Tienda'
ruta_lista_PRD= ruta_base + r'\\PRD_Tienda\\Lista_PRD'
ruta_lista_PICKUP = ruta_base + r'\\PRD_Tienda\\Lista_STORE_PICKUP'
# Opciones
## fechas
# Establecer la configuración regional en español
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
#fecha_hoy = pd.to_datetime('today').date()
fecha_hoy = pd.to_datetime('2024-06-10').date()
## formatear fecha
formato_hoy = fecha_hoy.strftime('%d.%m.%y')
## Warnings
warnings.filterwarnings(action='ignore')
# Configuración para deshabilitar la notación científica al mostrar el DataFrame
pd.set_option('display.float_format', None)
# Funciones
## Función para leer Excel más rápido

def read_excel(path: str, sheet_name: str) -> pd.DataFrame:
    buffer = StringIO()
    Xlsx2csv(path, outputencoding="utf-8", sheet_name=sheet_name).convert(buffer)
    buffer.seek(0)
    df = pd.read_csv(buffer)
    return df


SCOPES = ["https://www.googleapis.com/auth/drive"]

def download_sheet(service, sheet_id, output_filename):
    try:
        request = service.files().export_media(
            fileId=sheet_id,
            mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
        )
        file_data = io.BytesIO()
        downloader = MediaIoBaseDownload(file_data, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()

        with open(output_filename, 'wb') as f:
            f.write(file_data.getvalue())
            f.close()
        print(f"Google Sheets file '{sheet_id}' exported successfully as Excel (XLSX) format.")
    except HttpError as error:
        print(f"An error occurred while downloading sheet '{sheet_id}': {error}")

def main():
    creds = None
    API_SERVICE_NAME = 'drive'
    API_VERSION = 'v3'

    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)

        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build(API_SERVICE_NAME, API_VERSION, credentials=creds)

        # Descargar el primer archivo
        sheet_id_1 = 'url'
        download_sheet(service, sheet_id_1, os.path.join(ruta_lista_PRD,'Personal Recoleccion diaria '+  str(fecha_hoy)+'.xlsx')) ##str(pd.to_datetime('today').date())

        # Descargar el segundo archivo
        sheet_id_2 = 'url'
        download_sheet(service, sheet_id_2, 'Dato_tienda_Dev_C&C.xlsx')

    except HttpError as error:
        print(f"An error occurred: {error}")

if __name__ == "__main__":
    main()


In [ ]:
### reporte diario de urbano diario
ruta_PICKUP = glob.glob(os.path.join(ruta_lista_PICKUP,('Consolidado STORE PICK-UP '+ formato_hoy + '.xlsx')))
ruta_PICKUP_ = pd.DataFrame()
ruta_PICKUP_ = []  # Initialize an empty list instead of a DataFrame
x = pd.DataFrame()
for i in range(len(ruta_PICKUP)):
    x = pd.read_excel(ruta_PICKUP[i], 'CONSOLIDADO TOTAL', dtype={"num_rastreo": str})#,
    ruta_PICKUP_.append(x)
ruta_PICKUP_ = pd.concat(ruta_PICKUP_, ignore_index=True)

# 1. Rutas base
### Personal de recoleccion diaria
ruta_PRD = glob.glob(os.path.join(ruta_lista_PRD, ('Personal Recoleccion diaria ' + str(fecha_hoy) + '.xlsx')))
ruta_PRD_ = pd.DataFrame()
ruta_PRD_ = []  # Initialize an empty list instead of a DataFrame
x = pd.DataFrame()
for i in range(len(ruta_PRD)):
    x = pd.read_excel(ruta_PRD[i], 'Datos_Personal_Recojo', header=9)
    ruta_PRD_.append(x)  # Append each DataFrame to the list

ruta_PRD_ = pd.concat(ruta_PRD_, ignore_index=True)


### Datos de los tienda

ruta_TIENDAS = glob.glob(os.path.join(ruta_Tienda,"Dato_tienda_Dev_C&C.xlsx"))
ruta_TIENDAS_ = pd.DataFrame()
ruta_TIENDAS_ = []  # Initialize an empty list instead of a DataFrame

x = pd.DataFrame()
for i in range(len(ruta_TIENDAS)):
    x = pd.read_excel(ruta_TIENDAS[i], 'Datos',dtype={"CELULAR": str,"CELULAR_SEGUNDO_CONTACTO":str,"CELULAR_2":str})
    ruta_TIENDAS_.append(x)
ruta_TIENDAS_ = pd.concat(ruta_TIENDAS_, ignore_index=True)

# Quitar inconsistencias en nombres de columnas
ruta_PRD = ruta_PRD_.rename(columns=lambda x: x.strip()).copy()
ruta_PICKUP = ruta_PICKUP_.rename(columns=lambda x: x.strip()).copy()
ruta_TIENDAS = ruta_TIENDAS_.rename(columns=lambda x: x.strip()).copy()

## ruta_PRD
### Seleccionar columnas
ruta_PRD = ruta_PRD[['PICKUP', 'TIPO', 'PLACA', 'NOMBRES','DOCUMENTO','CARGO']]
### Cambiar los valores de las columnas
ruta_PRD.loc[ruta_PRD['PICKUP'] == 'MOY01', 'PICKUP'] = 'MOY IL 18-H1'
ruta_PRD.loc[ruta_PRD['PICKUP'] == 'MOY02', 'PICKUP'] = 'MOY IL 19-H1'

ruta_PRD.loc[ruta_PRD['PICKUP'] == 'SEDEL IL 01', 'PICKUP'] = 'SEDEL IL 01-PORTER'
ruta_PRD.loc[ruta_PRD['PICKUP'] == 'SEDEL IL 11', 'PICKUP'] = 'SEDEL IL 11-H1'
ruta_PRD.loc[ruta_PRD['PICKUP'] == 'SEDEL IL 12', 'PICKUP'] = 'SEDEL IL 12-H1'
ruta_PRD.loc[ruta_PRD['PICKUP'] == 'SEDEL IL 13', 'PICKUP'] = 'SEDEL IL 13-H1'


PICKUP_TIENDA = pd.merge(left=ruta_PICKUP ,right= ruta_TIENDAS , how='left', left_on='ID Tienda', right_on='COD_TIENDA' )

# Usando la función format()
PICKUP_TIENDA['num_rastreo'] = PICKUP_TIENDA['num_rastreo'].astype(str)
PICKUP_TIENDA['CORREOS'] = PICKUP_TIENDA['CORREOS'].astype(str)
print(PICKUP_TIENDA)

# Eliminar duplicados en la columna "Nombre"
PICKUP_TIENDA = PICKUP_TIENDA.drop_duplicates(subset=['numero_order'])
# Guardar el DataFrame como un archivo Excel
PICKUP_TIENDA.to_excel('datos.xlsx', index=False)

In [ ]:
import time
import datetime
import pandas as pd
##import pyautogui
import smtplib
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import math  # Necesario para verificar NaN
import os

# Obtener la fecha actual y calcular la semana correspondiente
week_number = fecha_hoy.strftime("%V")
dia_semana = fecha_hoy.strftime('%A')
archivo_registro = "Registro_Correos.xlsx"

# Crear un DataFrame vacío para almacenar el registro de envío de correos electrónicos
registro_correos = pd.DataFrame(columns=["Tienda", "Resultado"])

# Iterar sobre las tiendas únicas en el DataFrame
for tienda in PICKUP_TIENDA['Nombre_Tienda'].unique():
    df_tienda = PICKUP_TIENDA[PICKUP_TIENDA['Nombre_Tienda'] == tienda]
    if df_tienda.empty:
        print(f"No hay datos para la tienda {tienda}. No se enviará el correo.")
        continue
    correo_tienda = df_tienda['CORREOS'].iloc[0]
    ID_Tienda = df_tienda['ID Tienda'].iloc[0]
    if pd.isna(correo_tienda):
        print(f"La dirección de correo electrónico para la tienda {tienda} es nula (NaN). No se enviará el correo.")
        continue

    # Create a pivot table
    pivot_table = pd.pivot_table(df_tienda,
                             values=['numero_order'],
                             index=['Nombre_Tienda'],
                             columns=['Tipo Devolución','BU'],
                             aggfunc={'numero_order': 'nunique'}, #'count'
                             #margins=True,
                             #margins_name='Total'
                             ).rename(columns={'Tipo Devolución': 'Tipo'}
                                      ).rename_axis(columns={'Nombre_Tienda': 'Tienda'})

    # Rellenar los valores NaN con ''
    pivot_table.fillna('', inplace=True)

    # Create HTML representation of the styled pivot table
    html_table = pivot_table.to_html(classes='styled-table', escape=False)

    # Print the head of the pivot table
    Devo_df = df_tienda[df_tienda['Tipo Devolución'].str.contains('DEVOLUCIÓN', na=False) | df_tienda['Tipo Devolución'].str.contains('OFFLINE', na=False)]
    NoShow_df = df_tienda[df_tienda['Tipo Devolución'].str.contains('NO SHOW', na=False)]

    ##print(pivot_table)
    Devo_df= Devo_df[['numero_order','num_rastreo','ID Tienda',]]
    NoShow_df=NoShow_df[['numero_order','num_rastreo','ID Tienda',]]
    # Definir el nombre del archivo Excel con el formato deseado
    nombre_archivo = f"Devoluciones_{ID_Tienda}_W{week_number}.xlsx"

    # Escribir los DataFrames en un archivo Excel
    with pd.ExcelWriter(nombre_archivo) as writer:
        # Escribir 'filtered_rlos_tienda' en una hoja llamada 'filtered_rlos_tienda'
        Devo_df.to_excel(writer, sheet_name='Devolucion', index=False)
        NoShow_df.to_excel(writer, sheet_name='NoShow', index=False)
        workbook = writer.book
        Devo_sheet = writer.sheets['Devolucion']
        NoShow_sheet = writer.sheets['NoShow']

        # Formato para los títulos de fondo de color naranja
        header_format = workbook.add_format({'bg_color': '#a4d41e', 'bold': True})

        # Aplicar formato a los títulos en Backstore
        for col_num, value in enumerate(Devo_df.columns.values):
            Devo_sheet.write(0, col_num, value, header_format)
        Devo_sheet.autofilter(0, 0, len(Devo_df.index), len(Devo_df.columns) - 1)
        Devo_sheet.set_column(0, len(Devo_df.columns) - 1, 30)

        # Aplicar formato a los títulos en Offline
        for col_num, value in enumerate(NoShow_df.columns.values):
            NoShow_sheet.write(0, col_num, value, header_format)
        NoShow_sheet.autofilter(0, 0, len(NoShow_df.index), len(NoShow_df.columns) - 1)
        NoShow_sheet.set_column(0, len(NoShow_df.columns) - 1, 30)

    if not os.path.exists(nombre_archivo):
        print(f"El archivo {nombre_archivo} no fue creado correctamente.")
        registro_correos = pd.concat([registro_correos, pd.DataFrame({"Tienda": tienda, "Resultado": "Error: Archivo no creado"}, index=[0])])
        continue

    # Configuración
    email_usuario = 'CORREOS'
    contrasena = 'CONTRASEÑA'
    asunto_base = f"RECOLECCIÓN  // {tienda}// {fecha_hoy.year} - WEEK {week_number} - {fecha_hoy}"

    # Configuración de los servidores SMTP y los puertos
    servidores_smtp = 'smtp.office365.com'
    puertos_smtp = 587

    # Establecer la conexión con el servidor SMTP
    server = smtplib.SMTP(servidores_smtp, puertos_smtp)

    # Iniciar el cifrado TLS
    server.starttls()

    # Iniciar sesión en el servidor SMTP
    server.login(email_usuario, contrasena)
   # Dividir la cadena en una lista de direcciones de correo electrónico
    destinatarios_to = correo_tienda.split(',')

    # Concatenar las direcciones de correo electrónico en una cadena separada por comas
    destinatarios_str_to = ', '.join(destinatarios_to)

    # Concatenar las direcciones de correo electrónico en una cadena separada por comas
    destinatarios_CC = ['CORREOS']

    # Concatenar las direcciones de correo electrónico en una cadena separada por comas
    destinatarios_str_CC = ', '.join(destinatarios_CC)
    try:
        # Configurar el mensaje
        msg = MIMEMultipart()
        msg['From'] = 'CORREO'
        msg['To'] =  destinatarios_str_to #'mpazos@falabella.com' ##correo_tienda ##'mpazos@falabella.com'#'lshuan@falabella.com,ypalominop@falabella.com,mariortizb@falabella.com'
        msg['Subject'] = asunto_base
        msg['CC'] = destinatarios_str_CC

        # Darle un nivel de importancia al correo electrónico (en este caso, Alto)
        msg.add_header('Importance', 'High')

        # Crear el cuerpo del correo en formato HTML
        mensaje_html = f"""
<html>
  <head>
    <style>
      .styled-table th {{
        background-color: green;
        font-weight: bold;
      }}
    </style>
  </head>
  <body>
    <p>Estimado {tienda}!</p>
    <p>Se adjunta el detalle de los pedidos que serán recolectados el dia {dia_semana} bajo el nuevo flujo.</p>

    <p><span style="font-size: 16px"><u><strong> Cantidad de devoluciones cada tienda</strong></u></span><br></p>
    {html_table}
 <p>Quedo atento</p>
  <p>Saludos</p>
  <p>Equipo Home Delivery</p>
  </body>
</html>
"""

        # Adjuntar el cuerpo del correo en formato HTML
        msg.attach(MIMEText(mensaje_html, 'html'))

        # Adjuntar el archivo Excel al correo electrónico
        with open(nombre_archivo, "rb") as attachment:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', f"attachment; filename= {nombre_archivo}")
            msg.attach(part)

        # Enviar el correo electrónico
        server.sendmail(email_usuario,destinatarios_str_to.split(',') + destinatarios_str_CC.split(','), msg.as_string())
        registro_correos = pd.concat([registro_correos, pd.DataFrame({"Tienda": tienda, "Resultado": "Correo enviado correctamente"}, index=[0])])
        print(f"Correo electrónico enviado correctamente a {tienda}")

    except smtplib.SMTPDataError as e:
        registro_correos = pd.concat([registro_correos, pd.DataFrame({"Tienda": tienda, "Resultado": f"Error al enviar correo electrónico: {e}"}, index=[0])])
        print(f"Error al enviar correo electrónico a {tienda}: {e}")
    finally:
        server.quit()

# Guardar el registro de envío de correos electrónicos en un archivo Excel
registro_correos.to_excel(archivo_registro, index=False)
print(f"Se ha guardado el registro de envío de correos electrónicos: {archivo_registro}")